In [1]:
import pandas as pd
import numpy as np
import re
from gensim.parsing.preprocessing import remove_stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,Dropout,SpatialDropout1D
import warnings
warnings.filterwarnings('ignore')

In [142]:
# df = pd.read_csv('/content/Train.csv',on_bad_lines='skip',engine = 'python')

In [3]:
df = pd.read_csv('/kaggle/input/sentiments/Train.csv',on_bad_lines='skip',engine = 'python')

In [4]:
df.head(1)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0.0


In [5]:
df.shape

(40000, 2)

In [6]:
df['label'].value_counts(normalize=True)

label
0.0    0.500463
1.0    0.499537
Name: proportion, dtype: float64

In [7]:
df.isnull().sum()

text     1
label    1
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df['text'][0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [10]:
df['text'][66]

"First of all, I would like to say that I am a fan of all of the actors that appear in this film and at the time that I rented it, I wanted to like it.<br /><br />I think that the main reason that I was so disappointed was that the outside box promised me a suspense thriller. In my eyes, a suspense thriller for British movies is like something out of a Ruth Rendell novel, something that has a lot of dark twist and turns and leaves the viewer with an ending that is unlikely to be forgotten anytime soon.<br /><br />This movie started out with the promising note of being such a film. We have our main character, that suspects a man that he does not like, of being involved in a hit and run that killed the husband of one of his servants.His notions prove to be right, but the idea that his wife might be involved, does not occur to him until that she confesses to him that she was a part of the crime.<br /><br />The elements of a good suspense thriller were in place, at this point, but from the

In [11]:
html_pattern=re.compile(r'<[^>]+>')
non_alpha=re.compile(r'[^A-Za-z ]')

In [12]:
def preprocessing(data):
  #lower case
  review=data.lower()
  #remove html tags
  review=html_pattern.sub('',review)
  #remove stopwords
  review=remove_stopwords(review)
  #remove non alpha
  review=non_alpha.sub('',review)
  return review


In [13]:
df['text']=df['text'].apply(preprocessing)

In [14]:
df['text'][66]

'all like fan actors appear film time rented it wanted like iti think main reason disappointed outside box promised suspense thriller eyes suspense thriller british movies like ruth rendell novel lot dark twist turns leaves viewer ending unlikely forgotten anytime soonthis movie started promising note film main character suspects man like involved hit run killed husband servantshis notions prove right idea wife involved occur confesses crimethe elements good suspense thriller place point there felt film took different direction sort mild soap opera wants love real relationship is film enjoyable me outside box talked twisted lovers triangle labeled suspense thrillerthis soap opera story beginning setting mild distraction true content film felt like film lot better did felt like kept leading viewer big event materialized so lower rating liked fell short expectations'

In [157]:
# pip install tensorflow

In [18]:
tokenizer=Tokenizer(split=' ')
tokenizer.fit_on_texts(df['text'].values)

In [19]:
X=tokenizer.texts_to_sequences(df['text'].values)

In [20]:
len(tokenizer.word_index)

186436

In [30]:
# pip install tensorflow

In [159]:
# from keras.preprocessing.sequence import pad_sequences

In [22]:
max([len(i) for i in X])

1337

In [23]:
padded_X=pad_sequences(X)

In [24]:
len(padded_X[0])

1337

In [25]:
len(padded_X[67])

1337

In [26]:
y=df['label']

In [27]:
x_train,x_test,y_train,y_test=train_test_split(padded_X,y,test_size=0.2,random_state=11)

In [48]:
model=Sequential()

model.add(Embedding(186436,100,input_length=1337))
model.add(SpatialDropout1D(0.2))

model.add(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))

model.add(LSTM(32))

model.add(Dense(1,activation='sigmoid'))

In [49]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy','Precision','Recall'])

In [50]:
model.fit(x_train,y_train,batch_size=256,epochs=2,validation_data = (x_test, y_test))

Epoch 1/2
125/125 [==============================] - 515s 4s/step - loss: 0.4107 - accuracy: 0.8086 - precision: 0.8077 - recall: 0.8070 - val_loss: 0.2802 - val_accuracy: 0.8903 - val_precision: 0.8765 - val_recall: 0.9135
Epoch 2/2
125/125 [==============================] - 505s 4s/step - loss: 0.1551 - accuracy: 0.9445 - precision: 0.9438 - recall: 0.9445 - val_loss: 0.3035 - val_accuracy: 0.8844 - val_precision: 0.8728 - val_recall: 0.9051


In [ ]:
def prediction(review):
  preprocessed_text=preprocessing(review)
  tokenized_review=tokenizer.texts_to_sequences([preprocessed_text])
  padded_review=pad_sequences(tokenized_review,maxlen=1337)
  prob=model.predict(padded_review)
  if prob>0.5:
    return 'positive'
  else:
    return 'negative'

In [92]:
sent="""please don't watch this movie ,movie is so bad you will feel so bad if you watch the movie there is nothing
interested in this movie"""

In [154]:
prediction('very good movie must watch it once')

1/1 [==============================] - 0s 344ms/step


'positive'

In [99]:
# By using word2vec

In [110]:
from gensim.models import Word2Vec
sentences=[sent.split() for sent in df['text']]

In [101]:
len(sentences)

39999

In [107]:
clean_text=df['text'].apply(preprocess)

In [111]:
word2vec_model=Word2Vec(sentences,vector_size=100,window=5)

In [114]:

key_to_index = word2vec_model.wv.key_to_index


In [115]:
len(key_to_index)

39089

In [131]:
vector = word2vec_model.wv['movie']

In [132]:
len(vector)

100

In [127]:
import numpy as np
embedding_metrix=np.zeros((186436+1,100))
embedding_metrix.shape

(186437, 100)

In [135]:
for word,index in tokenizer.word_index.items():
    
    if word in word2vec_model.wv:
        
        embedding_metrix[index]=word2vec_model.wv[word]
    else:
        embedding_metrix[index]=np.zeros(100)

In [141]:
model=Sequential()

# model.add(Embedding(186436,100,input_length=1337))
model.add(Embedding(input_dim=186436+1, output_dim=100, weights=[embedding_metrix], input_length=1337, trainable=False))
model.add(SpatialDropout1D(0.2))

model.add(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))

model.add(LSTM(32))

model.add(Dense(1,activation='sigmoid'))

In [142]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy','Precision','Recall'])

In [143]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1337, 100)         18643700  
                                                                 
 spatial_dropout1d_1 (Spati  (None, 1337, 100)         0         
 alDropout1D)                                                    
                                                                 
 lstm_4 (LSTM)               (None, 1337, 64)          42240     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 18698389 (71.33 MB)
Trainable params: 54689 (213.63 KB)
Non-trainable params: 18643700 (71.12 MB)
________

In [144]:
model.fit(x_train,y_train,batch_size=256,epochs=2,validation_data = (x_test, y_test))

Epoch 1/2
125/125 [==============================] - 415s 3s/step - loss: 0.4683 - accuracy: 0.7753 - precision: 0.7607 - recall: 0.7994 - val_loss: 0.3669 - val_accuracy: 0.8447 - val_precision: 0.8228 - val_recall: 0.8865
Epoch 2/2
125/125 [==============================] - 405s 3s/step - loss: 0.3796 - accuracy: 0.8322 - precision: 0.8292 - recall: 0.8341 - val_loss: 0.3308 - val_accuracy: 0.8597 - val_precision: 0.8293 - val_recall: 0.9130


In [145]:
def prediction(review):
  preprocessed_text=preprocessing(review)
  tokenized_review=tokenizer.texts_to_sequences([preprocessed_text])
  padded_review=pad_sequences(tokenized_review,maxlen=1337)
  prob=model.predict(padded_review)
  if prob>0.5:
    return 'positive'
  else:
    return 'negative'

In [152]:
sent='good movie'

In [162]:
prediction(sent)


1/1 [==============================] - 0s 353ms/step


'positive'

In [148]:
sent_2="""please don't watch this movie ,movie is so bad you will feel so bad if you watch the movie there is nothing
interested in this movie"""

In [181]:
prediction(sent_2)

1/1 [==============================] - 0s 349ms/step


'negative'

In [182]:
sent='very good movie'

In [183]:
prediction(sent)

1/1 [==============================] - 0s 356ms/step


'positive'

In [ ]:
model.save('sentiment_model.model')